In [ ]:
import re
import sys
from itertools import chain
import numpy as np

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

In [ ]:
def get_objects(app, pattern, keep_out_of_service=False):
    objs = app.GetCalcRelevantObjects(pattern)
    if keep_out_of_service:
        return objs
    return [obj for obj in objs if not obj.outserv]

## Part I
First, we start the application and activate the desired project

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\Texas Grid'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

## Part II
Then, we collect all the objects that make up the network: at this stage, these are DIgSILENT objects.

#### Generators

In [ ]:
generators = get_objects(app, '*.ElmSym')
n_generators = len(generators)
print(f'There are {n_generators} generators.')

In [ ]:
slack = np.array([gen.ip_ctrl for gen in generators])
np.where(slack)[0]

In [ ]:
generators[319].loc_name

#### Loads

In [ ]:
loads = [load for load in get_objects(app, '*.ElmLod') if load.plini > 0 and load.qlini > 0]
n_loads = len(loads)
print(f'There are {n_loads} loads.')

#### Lines

In [ ]:
lines = get_objects(app, '*.ElmLne')
n_lines = len(lines)
print(f'There are {n_lines} lines.')

#### Transformers

In [ ]:
transformers = get_objects(app, '*.ElmTr2')
n_transformers = len(transformers)
print(f'There are {n_transformers} transformers.')

#### Static VAR systems

In [ ]:
SVSs = get_objects(app, '*.ElmSvs')
n_SVSs = len(SVSs)
print(f'There are {n_SVSs} static VAR systems.')

#### Storage

In [ ]:
storage = get_objects(app, '*.ElmStorage')
n_storage = len(storage)
print(f'There are {n_storage} storage elements.')

#### DSL objects
These are objects implemented using DIgSILENT Simulation Language (DSL)

In [ ]:
DSLs = get_objects(app, '*.ElmDsl')

AVRs = [dsl for dsl in DSLs if dsl.typ_id.loc_name[:4] == 'avr_']
GOVs = [dsl for dsl in DSLs if dsl.typ_id.loc_name[:4] == 'gov_']
PSSs = [dsl for dsl in DSLs if dsl.typ_id.loc_name[:4] == 'pss_']
DRPs = [dsl for dsl in DSLs if dsl.typ_id.loc_name[:4] == 'drp_']
power_plants = [comp_mod for comp_mod in app.GetCalcRelevantObjects('*.ElmComp')
                if 'Comp sym_' in comp_mod.loc_name]

n_DSLs = len(DSLs)
n_AVRs = len(AVRs)
n_GOVs = len(GOVs)
n_PSSs = len(PSSs)
n_DRPs = len(DRPs)
n_power_plants = len(power_plants)

if n_DSLs != n_AVRs + n_GOVs + n_PSSs + n_DRPs:
    raise Exception('Missing some DSL')

AVR_types = set([avr.loc_name for avr in AVRs])
GOV_types = set([gov.loc_name for gov in GOVs])
PSS_types = set([pss.loc_name for pss in PSSs])
DRP_types = set([drp.loc_name for drp in DRPs])

print(f'There are {n_power_plants} power plants.')

print(f'There are {n_DSLs} DSL objects subdivided among:')
print(f'   {n_AVRs} AVRs (types: ' + ', '.join(AVR_types) + ')')
print(f'   {n_GOVs} GOVs (types: ' + ', '.join(GOV_types) + ')')
print(f'   {n_PSSs} PSSs (type:  ' + ', '.join(PSS_types) + ')')
print(f'   {n_DRPs} DRPs (type:  ' + ', '.join(DRP_types) + ')')

There should be none of the following:

In [ ]:
shunts = get_objects(app, '*.ElmShnt')
capacitors = get_objects(app, '*.ElmScap')
impedances = get_objects(app, '*.ElmZpu')
n_shunts, n_capacitors, n_impedances = len(shunts), len(capacitors), len(impedances)
print(f'There are {n_shunts} shunts.')
print(f'There are {n_capacitors} series capacitors.')
print(f'There are {n_impedances} common impedances.')

In [ ]:
bus_names = []
for obj in chain(lines, capacitors, impedances):
    for i in 1,2:
        bus_name = obj.GetAttribute(f'bus{i}').cterm.loc_name
        if bus_name not in bus_names:
            bus_names.append(bus_name)
for obj in chain(generators, loads, shunts):
    bus_name = obj.bus1.cterm.loc_name
    if bus_name not in bus_names:
        bus_names.append(bus_name)
for obj in transformers:
    for c in 'hl':
        bus_name = obj.GetAttribute(f'bus{c}v').cterm.loc_name
        if bus_name not in bus_names:
            bus_names.append(bus_name)
all_in_service_buses = get_objects(app, '*.ElmTerm')
buses = [bus for bus in all_in_service_buses if bus.loc_name in bus_names]
n_buses = len(buses)
print(f'There are {n_buses} buses.')

## Part III
Here, we convert all the DIgSILENT objects to the corresponding Python class representation.
<br/>
But first, run a load flow analysis and make sure that there are no errors:

In [ ]:
load_flow = app.GetFromStudyCase('ComLdf')
err = load_flow.Execute()
if err:
    print('Error when running a load flow analysis.')
else:
    print('Successfully run a load flow analysis.')

In [ ]:
# shortcut function
get_name = lambda elem: elem.name

Group together all the objects that make up a powr plant, i.e., a synchronous machine, an AVR, a governor, a PSS and optionally a droop controller.

In [ ]:
class PowerSystemStabilizer (object):
    def __init__(self, pss, type_name):
        pass
    
class DroopController (object):
    def __init__(self, drp, type_name):
        pass

class TexasPowerPlant (object):
    def __init__(self, power_plant):
        self.name = power_plant.loc_name
        slots = power_plant.pblk
        elements = power_plant.pelm
        for slot,element in zip(slots, elements):
            if element is not None:
                element_name = element.loc_name
                if 'sym' in element_name:
                    self.gen = PowerGenerator(element.obj_id)
                else:
                    try:
                        type_name = element.typ_id.loc_name
                        if type_name[:4] == 'avr_':
                            self.avr = AutomaticVoltageRegulator(element, type_name=element_name)
                        elif type_name[:4] == 'gov_':
                            self.gov = TurbineGovernor(element, type_name=element_name)
                        elif type_name[:4] == 'pss_':
                            self.pss = PowerSystemStabilizer(element, type_name=element_name)
                        elif type_name[:4] == 'drp_':
                            self.droop = DroopController(element, type_name=elemnt_name)
                    except:
                        pass
        self.avr.vrating = self.gen.vrating
                    
    def __str__(self):
        return 'TexasPowerPlant'
#         bus_id = self.gen.bus_id
#         avr_str = self.avr.fmt.format(f'bus{bus_id}', f'avr{bus_id}')
#         if self.gov.type_name.upper() == 'IEEEG1':
#             gov_str = self.gov.fmt.format(f'php{bus_id}', f'omega{bus_id}')
#             gen_str = self.gen.fmt.format(f'avr{bus_id}', f'php{bus_id}')
#         elif self.gov.type_name.upper() == 'IEEEG3':
#             gov_str = self.gov.fmt.format(f'pm{bus_id}', f'omega{bus_id}')
#             gen_str = self.gen.fmt.format(f'avr{bus_id}', f'pm{bus_id}')
#         else:
#             raise Exception(f'Unknown governor type "{self.gov.type_name}"')
#         return avr_str + '\n\n' + gov_str + '\n\n' + gen_str

In [ ]:
plant = power_plants[0]
slots = plant.pblk
elements = plant.pelm
print('Slots:')
for i,slot in enumerate(slots):
    print(f'[{i+1:2d}] {slot.loc_name}')
print('Elements:')
for i,elem in enumerate(elements):
    if elem is not None:
        print(f'[{i+1:2d}] {elem.loc_name} -> ')

In [ ]:
gen = PowerGenerator(elements[0].obj_id)
print(gen)

In [ ]:
raise Exception('stop here')

In [ ]:
powerplants = []
for plant in power_plants:
    try:
        powerplants.append(TexasPowerPlant(plant))
    except:
        pass

powerplants = sorted(powerplants, key=lambda elem: elem.gen.name)
generators_in_plants_names = [plant.gen.name for plant in powerplants]

In [ ]:
powergenerators = sorted([PowerGenerator(gen) for gen in generators 
                          if gen.loc_name not in generators_in_plants_names], key=get_name)
powerloads = sorted([PowerLoad(load) for load in loads], key=get_name)
powerbuses = sorted([PowerBus(bus) for bus in buses], key=get_name)
powertransformers = sorted([PowerTransformer(transformer, voltages_from='bus') for transformer in transformers],
                           key=get_name)
powerlines = sorted([PowerLine(line) for line in lines], key=get_name)
# powershunts = sorted([Shunt(shunt) for shunt in shunts], key=get_name)
# powercapacitors = sorted([SeriesCapacitor(cap) for cap in capacitors], key=get_name)
# powerimpedances = sorted([CommonImpedance(imp) for imp in impedances], key=get_name)

## Part IV
Finally, we use the string representation of each Python object to write a Pan netlist.